In [ ]:
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from google.colab import files
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
!wget https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

In [ ]:
local_zip = '/content/rockpaperscissors.zip' 
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/tempDataset')
zip_ref.close()

In [ ]:
base_dir = '/content/tempDataset/rockpaperscissors/'
train_dir = os.path.join(base_dir, 'rps-cv-images')

In [ ]:
val_Split = 0.4
batch_Size = 32
target_Size = 150
mode = 'categorical'

In [ ]:
train_data = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    validation_split = val_Split 
)

validation_data = ImageDataGenerator(
    rescale = 1./255,
    validation_split = val_Split
)

In [ ]:
train_generator = train_data.flow_from_directory(
    train_dir,
    target_size = (target_Size, target_Size),
    batch_size = batch_Size, 
    class_mode = mode, 
    subset = 'training' 
)

validation_generator = validation_data.flow_from_directory(
    train_dir,
    target_size = (target_Size, target_Size),
    batch_size = batch_Size,
    class_mode = mode,
    subset = 'validation'
)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'Same', activation = 'relu', input_shape = (target_Size, target_Size, 3)))
model.add(MaxPooling2D(pool_size = (2,2))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'Same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2))) 
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'Same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2))) 
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'Same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2))) 
model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
# Callback
auto_reduction_LR = ReduceLROnPlateau(
    monitor = 'val_accuracy',
    patience = 2, #if after 2 epoch not improve reduce LR by factor
    verbose = 1,
    factor = 0.5,
    min_lr = 0.000003
)

auto_stop_learn = EarlyStopping(
    monitor = 'val_accuracy',
    min_delta = 0,
    patience = 4,
    verbose = 1,
    mode = 'auto' 
)

#Compile
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = Adam(learning_rate = 0.001),
    metrics = ['accuracy'] 
)

#Fit
History = model.fit(
    train_generator,
    epochs = 7,
    validation_data = validation_generator,
    callbacks = [auto_reduction_LR, auto_stop_learn]
)

In [ ]:
def eval_plot(History):
  plt.plot(History.history['loss'])
  plt.plot(History.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epochs')
  plt.legend(['train', 'test'])
  plt.show()

In [ ]:
def eval_score(model):
  validation_generator = train_data.flow_from_directory(
          train_dir, 
          target_size = (target_Size,target_Size), 
          batch_size= batch_Size, 
          class_mode = mode,
          shuffle = False,
          subset = 'validation'
  )

  num_test_samples = len(validation_generator.filenames)

  Y_test = model.predict(validation_generator, num_test_samples // batch_Size+1)
  y_test = np.argmax(Y_test, axis = 1)

  print('\nConfusion Matrix\n')
  print(confusion_matrix(validation_generator.classes, y_test))

  print('\nAccuracy Score: {:.2f}\n'.format(accuracy_score(validation_generator.classes, y_test)))

  print('Micro Precision: {:2f}'.format(precision_score(validation_generator.classes, y_test, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(validation_generator.classes, y_test, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(validation_generator.classes, y_test, average='micro')))

  print('Macro Precision: {:2f}'.format(precision_score(validation_generator.classes, y_test, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(validation_generator.classes, y_test, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(validation_generator.classes, y_test, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(validation_generator.classes, y_test, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(validation_generator.classes, y_test, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(validation_generator.classes, y_test, average='weighted')))

  print('\nClassification Report\n')
  target = ['Rock', 'Paper', 'Scissors']
  print(classification_report(validation_generator.classes, y_test, target_names=target))

In [ ]:
eval_plot(History)

In [ ]:
eval_score(model)

In [ ]:
def predict_image():
  uploaded = files.upload()

  for fn in uploaded.keys():
    #predict img
    path = fn
    img = image.load_img(path, target_size = (target_Size, target_Size))

    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])

    img_toNPArr = np.asarray(images)
    img_toNPArr *= (1./255)
    img_in = tf.reshape(img_toNPArr, shape = [1, target_Size, target_Size, 3])

    probability = round((sorted(model.predict(img_in)[0])[2])*100, 2)
    predict_class = np.argmax(model.predict(img_in))

    if predict_class == 0:
        predict_label = 'Paper'
    elif predict_class == 1:
        predict_label = 'Rock'
    else:
        predict_label = 'Scissor'
    
    print('\n')
    plt.show()
    print("\nImage prediction result: ", predict_label)
    print("Probability: ", probability, "%")
    print('\n')

In [ ]:
predict_image()